In [3]:
import tensorflow as tf
import pandas as pd
import yfinance as yf
import numpy as np
import math
from itertools import product, combinations
from sklearn.preprocessing import MinMaxScaler
from copy import copy
from collections import deque
from tqdm.auto import tqdm

In [4]:
class PortfolioEnv():
    def __init__(self, dates, datasets, n):
        self.dates = dates
        self.datasets = datasets
        self.n = n
        self.process_big_pt()
        
        self.initial_pt = 1000000 # 1 000 000
        self.c_minus = 0.0025 # 0.25%
        self.c_plus = 0.0025 # 0.25%
        self.delta = 10000 # 10 000
                
        self.process_actions()
        self.action_shape = self.actions.shape[0]
        self._episode_ended = False
    
    def reset(self):
        self.pt = self.initial_pt # 1 000 000
        self.wt = [0.25,0.25,0.25,0.25]
        
        self.current_tick = 1
        self.episode_ended = False
        
        return {'big_xt':np.array(self.big_pt[0,:,:]), 'wt_prime':self.wt}
    
    def process_actions(self):
        asset_number = 3
        action_number = 3

        seq = np.arange(asset_number)
        actions = []

        for c in product(seq, repeat=action_number):
            actions.append(c)

        self.actions = np.array(actions)
    
    def find_action_index(self,action):
        for ind, a in enumerate(self.actions):
            if np.array_equal(a, action):
                return ind;
    
    def process_big_pt(self):
        datasets = self.datasets
        date_start = self.dates[0]
        date_end = self.dates[1]

        dfs = []
        for d in datasets:
            ticker = yf.Ticker(d)
            # get historical market data
            df_ = ticker.history(start=date_start, end=date_end, interval="1d")    
            df_.rename(mapper={
                "Close": d+"_close",
                "Open": d+"_open",
                "High": d+"_high",
                "Low": d+"_low",
                "Volume": d+"_volume"
            }, inplace=True, axis=1)
            if "Dividends" in df_.columns:
                df_.drop(axis=1,labels=["Dividends", "Stock Splits"],inplace=True)
            dfs.append(df_)

        final_df = pd.concat(dfs, axis=1)
        final_df.dropna(inplace=True)
        self.final_df = final_df
        
        final_df = self.final_df
        n = self.n
        Pc = []
        for d in datasets:
            asset_close = final_df[d+"_close"].values
            asset_prev_close = final_df[d+"_close"].shift().values
            Kc = (asset_close - asset_prev_close) / asset_prev_close
            Kc = Kc[1:]
            Pc_ = [Kc[i:i+n] for i in range(len(asset_close)-n)]
            Pc.append(Pc_)
        Pc = np.array(Pc)
        Po = []
        for d in datasets:
            asset_prev_close = final_df[d+"_close"].shift().values
            asset_open = final_df[d+"_open"].values
            Ko = (asset_open - asset_prev_close) / asset_prev_close
            Ko = Ko[1:]
            Po_ = [Ko[i:i+n] for i in range(len(asset_open)-n)]
            Po.append(Po_)
        Po = np.array(Po)
        Pl = []
        for d in datasets:
            asset_close = final_df[d+"_close"].values
            asset_low = final_df[d+"_low"].values
            Kl = (asset_close - asset_low) / asset_low
            Kl = Kl[1:]
            Pl_ = [Kl[i:i+n] for i in range(len(asset_low)-n)]
            Pl.append(Pl_)
        Pl = np.array(Pl)
        Ph = []
        for d in datasets:
            asset_close = final_df[d+"_close"].values
            asset_high = final_df[d+"_high"].values
            Kh = (asset_close - asset_high) / asset_high
            Kh = Kh[1:]
            Ph_ = [Kh[i:i+n] for i in range(len(asset_high)-n)]
            Ph.append(Ph_)
        Ph = np.array(Ph)
        Pv = []
        for d in datasets:
            asset_prev_volume = final_df[d+"_volume"].shift().values
            asset_volume = final_df[d+"_volume"].values
            Kv = (asset_volume - asset_prev_volume) / asset_prev_volume
            Kv = Kv[1:]
            Pv_ = [Kv[i:i+n] for i in range(len(asset_high)-n)]
            Pv.append(Pv_)
        Pv = np.array(Pv)
        Pt_star = np.array([Pc, Po, Pl, Ph, Pv])

        self.big_pt = Pt_star.swapaxes(0,2).swapaxes(1,2)
        print(self.big_pt.shape)
    
    def phi(self,v):
        return np.insert(v, 0, [0]) + np.ones(len(v) + 1)
    
    def get_wt_prime_chapeau(self,wt_prime,big_s_minus,big_s_plus,pt_prime):
        wt_prime_chapeau = []
        for ind, key in enumerate(wt_prime):
            if(ind in big_s_minus):
                wt_prime_chapeau.append(key - self.delta/pt_prime)
            elif(ind in big_s_plus):
                wt_prime_chapeau.append(key + self.delta/pt_prime)
            else:
                wt_prime_chapeau.append(key)
    
        return np.array(wt_prime_chapeau)
    
    def is_asset_shortage(self,action,pt,wt):
        big_s_minus = np.where(action==0)[0]
        for ind in big_s_minus:
            if wt[ind+1]*pt < self.delta:
                return True
        
        return False
    
    def is_cash_shortage(self,action,pt,wt):
        big_s_minus = np.where(action==0)[0]
        big_s_plus = np.where(action==2)[0]
        current_cash = wt[0]*pt
        cash_after_selling = current_cash + (1-self.c_minus)*self.delta*len(big_s_minus) # must include transaction costs
        cash_needed = (self.c_plus+1)*self.delta*len(big_s_plus) # must include transaction costs
        
        if(cash_after_selling < cash_needed):
            return True
        
        return False
    
    def action_mapping(self,action,action_Q_values,pt,wt):
        action = copy(action)
        action_mapped = action
        if self.is_asset_shortage(action,pt,wt):
            action_mapped = self.rule2(action,action_Q_values,pt,wt)
        elif self.is_cash_shortage(action,pt,wt):
            action_mapped = self.rule1(action,action_Q_values,pt,wt)
        
        return action_mapped
    
    def rule1(self,action,action_Q_values,pt,wt):
        MAXQ = np.NINF
        action_selected = action
        
        big_s_plus = np.where(action==2)[0]
        
        for i in range(1,len(big_s_plus) + 1):
            for c in combinations(big_s_plus, i):
                new_action = copy(action)
                for j in c:
                    new_action[j] = 1

                if not self.is_cash_shortage(new_action,pt,wt):
                    new_action_index = self.find_action_index(new_action)
                    new_action_Q_value = action_Q_values[new_action_index]
                    try:
                        if new_action_Q_value > MAXQ:
                            MAXQ = new_action_Q_value
                            action_selected = new_action
                    except ValueError:
                        print(':::::: ERROR ::::::')
                        print(action_Q_values.shape)
                        print(new_action_index)
                        print(new_action)
                        print(new_action.shape)
                        print(self.actions)
        
        return action_selected
    
    def rule2(self,action,action_Q_values,pt,wt):
        for i in range(len(action)):
            if wt[i+1]*pt < self.delta:
                action[i] = 1
        
        if self.is_cash_shortage(action,pt,wt):
            action = self.rule1(action,action_Q_values,pt,wt)
        
        return action
    
    def F(self,pt,wt):
        action_possible = []
        for ind, action in enumerate(self.actions):
            if not self.is_asset_shortage(action,pt,wt) and not self.is_cash_shortage(action,pt,wt):
                action_possible.append(ind)

        return np.array(action_possible)
    
    def step(self, action, simulation=False):
        # Must set new portfolio with regards to action
        # Must set new reward
        if self.current_tick == len(self.big_pt) - 2:
            # The last action ended the episode. Ignore the current action and start
            # a new episode.
            self.episode_ended = True
        
        ktc = self.big_pt[self.current_tick,0,:,9] # nouveaux close prices des différents assets
        pt_prime = self.pt * np.dot(self.wt,self.phi(ktc)) # nouvelle valeur du portfolio issue de l'action précédente
        
        wt_prime = (self.wt*self.phi(ktc)) / (np.dot(self.wt,self.phi(ktc))) # nouvelles proportions des assets du portfolio
        
        # On prend en compte la nouvelle action
        big_s_minus = np.where(action==0)[0]
        big_s_plus = np.where(action==2)[0]
        
        ct = (self.delta*(self.c_minus*len(big_s_minus) + self.c_plus*len(big_s_plus)))/pt_prime
        if not simulation:
            self.pt = pt_prime*(1 - ct)
        else:
            pt = pt_prime*(1 - ct)
        
        wt_prime_chapeau_1tillend = self.get_wt_prime_chapeau(wt_prime[1:],big_s_minus,big_s_plus,pt_prime)
        wt_prime_chapeau_0 = wt_prime[0] + self.delta*((1-self.c_minus)*len(big_s_minus)-(1+self.c_plus)*len(big_s_plus))/pt_prime
        wt_prime_chapeau = np.concatenate((np.array([wt_prime_chapeau_0]), wt_prime_chapeau_1tillend))
        
        if not simulation:
            self.wt = wt_prime_chapeau / (np.dot(wt_prime_chapeau, np.ones(len(wt_prime_chapeau))))
            self.current_tick += 1
            k_t_plus_one_c = self.big_pt[self.current_tick,0,:,9]
        else:
            wt = wt_prime_chapeau / (np.dot(wt_prime_chapeau, np.ones(len(wt_prime_chapeau))))
            current_tick = self.current_tick + 1
            k_t_plus_one_c = self.big_pt[current_tick,0,:,9]
        
        big_p_s_t_plus_one = pt_prime*np.dot(wt_prime, self.phi(k_t_plus_one_c))
        
        if not simulation:
            p_t_plus_one_prime = self.pt * np.dot(self.wt,self.phi(k_t_plus_one_c))
            reward = (p_t_plus_one_prime - big_p_s_t_plus_one)/big_p_s_t_plus_one
            wt_plus_one_prime = (self.wt*self.phi(k_t_plus_one_c)) / (np.dot(self.wt,self.phi(k_t_plus_one_c)))
        else:
            p_t_plus_one_prime = pt * np.dot(wt,self.phi(k_t_plus_one_c))
            reward = (p_t_plus_one_prime - big_p_s_t_plus_one)/big_p_s_t_plus_one
            wt_plus_one_prime = (wt*self.phi(k_t_plus_one_c)) / (np.dot(wt,self.phi(k_t_plus_one_c)))
        
        if not simulation:
            return {'big_xt':np.array(self.big_pt[self.current_tick]), 'wt_prime':wt_plus_one_prime}, reward, self.episode_ended
        else:
            return {'big_xt':np.array(self.big_pt[current_tick]), 'wt_prime':wt_plus_one_prime}, reward, self.episode_ended, pt, wt

In [5]:
datasets = ["SPY", "IWD", "IWC"]
train_dates = ["2010-01-01", "2016-12-30"]
n = 10
env = PortfolioEnv(train_dates, datasets, 10)
datas_ = env.big_pt
datas = datas_.swapaxes(2,3).swapaxes(1,3)
final_datas = []
for d in datas:
    final_datas.append(d[0])
    final_datas.append(d[1])
    final_datas.append(d[2])
final_datas = np.array(final_datas)
mm_scaler = MinMaxScaler()
datas_scaled = np.array([mm_scaler.fit_transform(d) for d in final_datas]) # MinMaxScaler par ligne ou par colonne ?
X_train = datas_scaled[:4500]
X_valid = datas_scaled[4500:]

(1751, 5, 3, 10)


In [ ]:
recurrent_encoder = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(units=128, input_shape=[10, 5], return_sequences=True),
    tf.keras.layers.LSTM(units=10),
])
recurrent_decoder = tf.keras.models.Sequential([
    tf.keras.layers.RepeatVector(10, input_shape=[10]),
    tf.keras.layers.LSTM(units=128, return_sequences=True),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(5, activation="sigmoid"))
])
recurrent_ae = tf.keras.models.Sequential([recurrent_encoder, recurrent_decoder])
recurrent_ae.compile(optimizer='adam', loss='binary_crossentropy')
recurrent_encoder.summary()
recurrent_decoder.summary()
recurrent_ae.summary()

history = recurrent_ae.fit(X_train, X_train, epochs=40, validation_data=(X_valid, X_valid))

In [7]:
def create_envs(dates, datasets, n):
    return np.array([PortfolioEnv(d, datasets, n) for d in dates])
datasets = ["SPY", "IWD", "IWC"]
train_dates = [
    ["2010-01-01", "2010-12-30"],
    ["2011-01-01", "2011-12-30"],
    ["2012-01-01", "2012-12-30"],
    ["2013-01-01", "2013-12-30"],
    ["2014-01-01", "2014-12-30"],
    ["2015-01-01", "2015-12-30"],
    ["2016-01-01", "2016-12-30"],
]
n = 10
train_envs = create_envs(train_dates, datasets, 10)

(240, 5, 3, 10)
(241, 5, 3, 10)
(239, 5, 3, 10)
(240, 5, 3, 10)
(240, 5, 3, 10)
(240, 5, 3, 10)
(241, 5, 3, 10)


In [8]:
@tf.function
def train_step(preprocessed_states, mask, target_Q_values):
    with tf.device('gpu:0'):
        with tf.GradientTape() as tape:
            all_Q_values = model(preprocessed_states)
            Q_values = tf.reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
            loss = tf.reduce_mean(loss_fn(target_Q_values, Q_values))
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [9]:
beta = .3
epochs = 500 

bins = np.arange(0, 20, 1)
r = np.random.rand(epochs)
N=len(train_envs)
gen_trunc=(N-1-np.floor(np.log(1-r*(1-(1-beta)**N))/np.log(1-beta))).astype(int)

In [10]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

K = tf.keras.backend

input_states = tf.keras.layers.Input(shape=[34])
hidden1 = tf.keras.layers.Dense(64, activation="relu")(input_states)
hidden2 = tf.keras.layers.Dense(32, activation="relu")(hidden1)
output = tf.keras.layers.Dense(train_envs[0].action_shape)(hidden2)

model = tf.keras.models.Model(inputs=[input_states], outputs=[output])

target = tf.keras.models.clone_model(model)
target.set_weights(model.get_weights())

In [11]:
def preprocess_state(state, concatenate=True): # works when only one state, but not with several states
    big_xt = state['big_xt']
    big_xt = big_xt.swapaxes(0,1).swapaxes(1,2)
    big_xt_scaled = np.array([mm_scaler.fit_transform(a) for a in big_xt])

    predictions = recurrent_encoder.predict(big_xt_scaled)
    predictions_reshaped = predictions.reshape((big_xt_scaled.shape[0]*big_xt_scaled.shape[1]))
    
    if(concatenate):
        predictions_reshaped = np.concatenate((predictions_reshaped, state['wt_prime']))
    
    return predictions_reshaped

In [ ]:
replay_memory = deque(maxlen=2000)

epsilon = 1
batch_size = 16
discount_rate = .9

optimizer = tf.keras.optimizers.Adam(lr=1e-7)
loss_fn = tf.keras.losses.mean_squared_error

steps = 0
episode_counter = 1

train_CR = []
previous_CR = 0

for ind in gen_trunc:
    env = train_envs[ind]
    state = env.reset()
        
    pbar = tqdm(total=len(env.big_pt)-1)
    
    epsilon = max(1 - episode_counter / 400, 0.01)
    
    while 1: # will break when end of env
        pbar.set_description(f'episode {episode_counter}/{len(gen_trunc)} - previous CR:{previous_CR}')
        #epsilon-greedy policy to select an action
        action = None
        state_preprocessed = preprocess_state(state)
        if np.random.rand() < epsilon: # goal is to take uniformly random action among possible actions of this env
            possible_actions = env.F(env.pt, env.wt)
            random_index = np.random.randint(len(possible_actions))
            action = env.actions[possible_actions[random_index]]
        else: # here goal is to make model predict best action, but if action isn't feasable for env, map it
            Q_values = model.predict(state_preprocessed[np.newaxis])
            action = env.action_mapping(env.actions[np.argmax(Q_values[0])], Q_values[0], env.pt, env.wt)
        
        # simulate all feasible actions for current state
        possible_actions = env.F(env.pt, env.wt)
        simulations = []
        
        next_state_simulated_preprocessed_partial = None
        is_next_state_simulated_preprocessed_partial = False
        for simulated_action in possible_actions:
            next_state_simulated, next_reward, done, next_pt, next_wt = env.step(env.actions[simulated_action], simulation=True)
            if not is_next_state_simulated_preprocessed_partial:
                next_state_simulated_preprocessed_partial = preprocess_state(next_state_simulated, concatenate=False)
                is_next_state_simulated_preprocessed_partial = True
                
            next_state_simulated_preprocessed = np.concatenate((next_state_simulated['wt_prime'], next_state_simulated_preprocessed_partial))
            simulations.append((state_preprocessed, simulated_action, next_reward, next_state_simulated_preprocessed, done, next_pt, next_wt))
        replay_memory.append(np.array(simulations))
        next_state, reward, episode_ended = env.step(action)
        # do some stuff with it
        
        # add some time for buffer to populate
        
        if steps >= 50:
            # start batch training (room to improvment, why don't use Prioritized Experience Replay ?)
            # select a uniformly random batch
            indices = np.random.randint(len(replay_memory), size=batch_size)
            batch = [replay_memory[index] for index in indices]
            for simulations in batch:
                #start = time.time()
                preprocessed_states, actions, rewards, preprocessed_next_states, dones, next_pts, next_wts = [np.array([simulation[field_index] for simulation in simulations]) for field_index in range(7)]
                
                next_Q_values = target.predict(preprocessed_next_states) # Q-values predicted for all next_states
                max_next_Q_values_index = np.argmax(next_Q_values, axis=1) # for each next_states find index of max Q-value predicted by target
                
                # okay, here, in max_next_Q_values, there are some actions that may not be feasible and that we need to map
                # but we have to use is_asset_shortage, is_cash_shortage and action_mapping with state of simulation
                max_next_Q_values_actions = np.array([
                    env.actions[action_simulated] if not env.is_asset_shortage(env.actions[action_simulated], next_pts[ind], next_wts[ind]) and not env.is_cash_shortage(env.actions[action_simulated], next_pts[ind], next_wts[ind])
                    else env.action_mapping(env.actions[action_simulated], next_Q_values[ind], next_pts[ind], next_wts[ind])
                    for ind, action_simulated in enumerate(max_next_Q_values_index)
                ]) # here mistake
                
                # okay, we have the feasible actions selected by the target network, now we need to find their indexes to communicate with the network
                max_next_Q_values_indexes = [env.find_action_index(action_simulated) for action_simulated in max_next_Q_values_actions]
                # now we have to find the Q-values predicted by target network of these corrected actions
                max_next_Q_values = [next_Q_values[ind][best_action_simulated] for ind, best_action_simulated in enumerate(max_next_Q_values_indexes)]

                # and now, we can proprely calculate the target Q-value
                target_Q_values = (rewards +
                                   (1 - dones) * discount_rate * max_next_Q_values) # here problem with dones /!\
                target_Q_values = target_Q_values.reshape(-1, 1)
                mask = tf.one_hot(actions, env.action_shape)
                
                train_step(preprocessed_states, mask, target_Q_values)
                # and we're done, amazing !!
                #print('updates ::: {:.5f} ms / step'.format((time.time() - start) * 1000))
            
        state = next_state

        steps += 1
        pbar.update(1)
        
        if episode_ended:
            break;
        
    episode_counter += 1
    
    previous_CR = round((env.pt-env.initial_pt)/env.initial_pt, 3)
    train_CR.append(previous_CR)
    pbar.close()
    # update of target network
    target.set_weights(model.get_weights()) 